<a href="https://colab.research.google.com/github/salemsama/nn-zero-to-hero/blob/main/makemore1/makemore_part1_bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
words = open("names.txt", 'r').read().splitlines()

chars = sorted(list(set(''.join(words))))       # all letters, set used first because it doesn't allow duplicate
stoi = {s:i+1 for i,s in enumerate(chars)}      # dict to map string to int,    start from 1, 0 reserved for next line
stoi['.'] = 0                                   # special letter to mark start and end of word
itos = {i:s for s,i in stoi.items()}            # dict to map int to string


xs, ys = [], [] # lists for first letters and following letters
for word in words:
    chs = ['.'] + list(word) + ['.']    # word was string now list of char and special char before and after
    for ch1,ch2 in zip(chs, chs[1:]):   # zip make a tuple of every couple of chs
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]                 # store the number of the char

        xs.append(ix1)                  # here map xs ans ys
        ys.append(ix2)                  # xs is the input and ys is the correct output
        ...
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

228146

In [ ]:
# initialize the 'network'
gen = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=gen, requires_grad=True) # 27*27 network with grad

In [ ]:
def grad_desc(n):
    for k in range(n):
        # forward pass
            # we reprsent every letter as one hot (arrays of 0 except 1 place of the letter index)
        xenc = F.one_hot(xs, num_classes=27).float() # xs encoded as noe hots
        #one hot for a for example
        #[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

        # the input * the weights
        logits = xenc @ W # matrix mulipication in pytorch, logits for log-counts
        # we mulitply xenc wich is n*27 @ 27*27 it out put n*27
        # thats why W is 27*27 in order for the mulipication to be correct
        # if it was 27*1 it would give us n*1 output

        # non linearity, this two lines called softmax
        counts = logits.exp() # fake counts, when exp all numbers become positive kinda like counts
        probs = counts / counts.sum(1, keepdim=True) # normalizing the probability so the sum equals 1
        # the division here is 27*27 / 27*1 so every row is divided by its sum

        # the loss function
        loss= probs[torch.arange(num), ys] # here we collect the probabilities of the correct output ys
        loss= -loss.log().mean()    # here we used the negative liklyhood loss function
        loss = loss + 0.01 * (W**2).mean() # regularazition, like adding 1 to counts to there is no 0 count
        print(loss.item())

        # backward pass
        W.grad = None   #zerograd, None is more efficient than zero
        loss.backward() # set the grad to how much it influnce the loss
        W.data += -50*W.grad # we use the minus becuase we want to lower the loss

In [ ]:
grad_desc(100)

3.7686190605163574
3.3788065910339355
3.16109037399292
3.0271859169006348
2.9344842433929443
2.867231607437134
2.8166542053222656
2.777146339416504
2.7452542781829834
2.7188303470611572
2.696505546569824
2.6773719787597656
2.6608052253723145
2.6463515758514404
2.633664846420288
2.622471570968628
2.6125476360321045
2.6037068367004395
2.595794916152954
2.5886809825897217
2.5822560787200928
2.5764293670654297
2.5711236000061035
2.5662729740142822
2.5618228912353516
2.5577263832092285
2.5539441108703613
2.550442695617676
2.5471925735473633
2.5441696643829346
2.5413522720336914
2.538722038269043
2.536262035369873
2.5339581966400146
2.531797409057617
2.529768228530884
2.527859926223755
2.5260636806488037
2.5243701934814453
2.522773265838623
2.52126407623291
2.519836902618408
2.5184857845306396
2.5172054767608643
2.515990734100342
2.5148372650146484
2.5137407779693604
2.512697696685791
2.511704921722412
2.5107581615448
2.509854555130005
2.5089924335479736
2.5081682205200195
2.507380485534668


In [ ]:
for i in range(5):
            out = []
            ix = 0
            while True:
                xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
                logits = xenc @ W
                counts = logits.exp()
                p = counts / counts.sum(1, keepdim=True)

                ix = torch.multinomial(p,num_samples=1,replacement=True,generator=gen).item()
                out.append(itos[ix])
                if ix == 0:
                    break
            print(''.join(out))

morvann.
akela.
az.
arileri.
chaiadayra.
